# Drugi etap projektu
Julia Jodczyk

Filip Pawłowski 
### Polecenie:
“Jakiś czas temu wprowadziliśmy konta premium, które uwalniają użytkowników od słuchania reklam. Nie są one jednak jeszcze zbyt popularne – czy możemy się dowiedzieć, które osoby są bardziej skłonne do zakupu takiego konta?”

In [1]:
import pickle
import requests
import json
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from microservice.load_data import Preprocessor, DataModel
from microservice.files_utils import randomly_split_group

## Modele

Stworzyliśmy modele klasyfikacji binarnej, które dzielą użytkowników na grupy: `kupi premium` i `nie kupi premium`. 

### Model Bazowy

Jako model bazowy, najprostszy z możliwych dla danego zadania uznajemy model naiwny, który zawsze klasyfikuje użytkowników do grupy `kupi premium`: 

In [2]:
class NaiveModel:
    def predict(self, input_df):
        user_ids = input_df.index
        mock_series = pd.Series(True, index=user_ids, name="user_id")
        return mock_series
    
base_model = NaiveModel()

with open('./microservice/saved_models/base_model.sav', 'wb') as f:
    pickle.dump(base_model, f)

### Model docelowy

Jako model docelowy, po analizie z pierwszego etapu, wybraliśmy model KNN z następującymi cechami (per użytkownik):
- miasto 
- stosunek czasu reklam do całego czasu, jaki użytkownik spędził korzystając z serwisu
- stosunkowy udział każdego typu zdarzenia (event_type) we wszystkich zdarzeniach sesji
- stosunek ilości reklam po utworach ulubionego gatunku
- ulubione gatunki użytkownika

Implementacja ekstrakcji powyższych cech została umieszczona w pliku `load_data.py`. Cechy nieliczbowe - miasto oraz ulubione gatunki zostały zakodowane sposobem one hot encoding.

In [3]:
with open("./data/users.json", "r") as f:
        group = json.load(f)
train_users, test_users = randomly_split_group(group)
with open("./data/train_users.json", "w") as f:
    json.dump(train_users, f)

In [4]:
target_model = KNeighborsClassifier()
# load data:
# use file train_users instead of users
data_model = DataModel()
data_model.users_df = pd.read_json("./data/train_users.json")
df = data_model.get_merged_dfs()
preprocessed_df = Preprocessor.run(df)

In [5]:
preprocessed_df

,user_id,premium_user,Gdynia,Kraków,Poznań,Radom,Szczecin,Warszawa,Wrocław,Ads_ratio,...,ranchera,regional mexican,rock,rock en espanol,roots rock,singer-songwriter,soft rock,soul,tropical,vocal jazz
0,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
1,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
2,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
3,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
4,101,True,0,0,0,0,0,0,1,0.030566,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184878,1054,True,0,0,0,0,1,0,0,0.027753,...,0,0,0,0,1,0,0,0,0,0
184879,1054,True,0,0,0,0,1,0,0,0.027753,...,0,0,0,0,1,0,0,0,0,0
184880,1054,True,0,0,0,0,1,0,0,0.027753,...,0,0,0,0,1,0,0,0,0,0
184881,1054,True,0,0,0,0,1,0,0,0.027753,...,0,0,0,0,1,0,0,0,0,0


In [8]:
# split data:
X, y = preprocessed_df.drop(["premium_user"], axis=1), preprocessed_df["premium_user"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=18)

target_model.fit(X_train, y_train)

with open('./microservice/saved_models/KNN_model.sav', 'wb') as f:
    pickle.dump(target_model, f)

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [8]:
base_y_hat = base_model.predict(X_offline_test)
base_y_hat

166587    True
91690     True
122185    True
77859     True
135832    True
          ... 
1932      True
184633    True
169210    True
165052    True
129575    True
Name: user_id, Length: 61012, dtype: bool

In [6]:
score = accuracy_score(y_offline_test, base_y_hat)
score 

NameError: name 'base_y_hat' is not defined

In [9]:
target_y_hat = target_model.predict(X_test)
score = accuracy_score(y_test, target_y_hat)
score 

/home/julia/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


1.0

### Porównanie wyników offline

In [11]:
base_test_online, target_test_online = randomly_split_group(test_offline)

y_hat = base_model.predict(base_test_online)

score = accuracy_score(y_test, y_hat)
score 

NameError: name 'test_offline' is not defined

### Porównanie wyników

Wyniki predykcji zbierzemy za pomocą zaimplementowanego mikroserwisu (szczegóły implementacji i API niżej). 

(Przed uruchomieniem kodu z poniższej komórki należy uruchomić mikroserwis komendą `python3 /microservice/microservice.py`) 

In [ ]:
base_url = "http://127.0.0.1:8000"

base_model_test, target_model_test = randomly_split_group(test_users)
for user in target_model_test:
    requests.post(f"{base_url}/predict-with/KNN", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)
for user in base_model_test:
    requests.post(f"{base_url}/predict-with/base", params={"test": "True"}, json=user)
    actual_body = {
        "user_id": user["user_id"],
        "actual": user["premium_user"]
    }
    requests.post(f"{base_url}/submit-actual", json=actual_body)


In [ ]:
response = requests.get(f"{base_url}/test_ab_results")


### Dalsze kroki:
- opis mikroserwisu
- analiza wyników testów